### 1. Подключение библиотек
Наличие большого числа разносторонних библиотек - одно из ключевых преимуществ языка Python  
Среди огромного количества библиотек стоит выделить следующие:
* numpy - библиотека для работы с массивами/матрицами
* pandas - библиотека для работы с таблицами
* RDKit - библиотека для работы с молекулами и хим. сущностями
* matplotlib - библиотека для визуализации
* psi4 - квантовые расчеты
* openmm - мол. динамика
* pickle - библиотека для сохранения объектов в бинарных файлах
* os - для работы с файловой системой

In [ ]:
### Pandas понадобится нам для работы с данными
import pandas as pd
### Различные подмодули из rdkit
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from rdkit.Chem import rdFMCS
from rdkit.Chem.Draw import IPythonConsole
### для сохранения в файл
import pickle
### для работы с файловой системой
import os
### для отрисовки
from matplotlib import pyplot as plt
from matplotlib.image import NonUniformImage
### Для работы с массивами
import numpy as np
### Импорты для анализа потраченной памяти
from sys import getsizeof
import psutil

### Переменная, указывающая папку с нашими файлами
data_folder = "/home/alex/ml_lectures"

#### Взаимодествие с ОС
1. Посредством библиотеки os # Python-like
2. Посредством вызова комманд оболочки из ячейки

### 2. Загрузка исходных данных
#### Что представляют данные в датасете
    I.  Property  Unit         Description
    --  --------  -----------  --------------
     1  tag       -            "gdb9"; string constant to ease extraction via grep
     2  index     -            Consecutive, 1-based integer identifier of molecule
     3  A         GHz          Rotational constant A
     4  B         GHz          Rotational constant B
     5  C         GHz          Rotational constant C
     6  mu        Debye        Dipole moment
     7  alpha     Bohr^3       Isotropic polarizability
     8  homo      Hartree      Energy of Highest occupied molecular orbital (HOMO)
     9  lumo      Hartree      Energy of Lowest occupied molecular orbital (LUMO)
    10  gap       Hartree      Gap, difference between LUMO and HOMO
    11  r2        Bohr^2       Electronic spatial extent
    12  zpve      Hartree      Zero point vibrational energy
    13  U0        Hartree      Internal energy at 0 K
    14  U         Hartree      Internal energy at 298.15 K
    15  H         Hartree      Enthalpy at 298.15 K
    16  G         Hartree      Free energy at 298.15 K
    17  Cv        cal/(mol K)  Heat capacity at 298.15 K

#### __2.1 Из таблицы csv / xlsx__

In [ ]:
### Ищем в папке все, где есть qm9
### Посредством библиотеки os # Python-like
### функция .lower() применеятся, чтобы поиск 
### был нечуствителен к регистру (ключ -i в grep)
print([filename for filename in os.listdir(data_folder) if "qm9" in filename.lower()])
print()
### Посредством вызова комманд оболочки из ячейки
### Отметим, что data_folder передана как переменная
!ls $data_folder | grep -i qm9

In [ ]:
### Напрямую загружаем таблицу из csv файла, также можно работать и с xlsx файлами
print("Время чтения из csv файла:")
### os.path.join корректно объединяет название папок и файлов, 
###   чтобы не заморачиваться со слэшами
%time df1 = pd.read_csv(os.path.join(data_folder,"qm9.csv"))
### %time - это фишка jupyter, позволяет замерить время выполнения кода в строке
### %%time в начале ячейки позволяет замерить ячейку целиком
print("\nВремя чтения из xlsx файла:")
%time df2 = pd.read_excel(os.path.join(data_folder,"qm9.xlsx"))

### Посмотрим на размерность и проверим таблицы на идентичность
if df1.equals(df2): print("\nМатрицы из csv и xlsx файла идентичны ") 
else: raise Exception("Матрицы не равны")
print("\nРазмер исходной таблицы:\n{} строк и {} столбцов".format(df1.shape[0],df1.shape[1]))

__ВЫВОД__: Не храните большие данные в Excel-формате.

#### __2.2 Из бинарного pickle файла__

In [ ]:
with open(os.path.join(data_folder,"qm9.pkl"), 'wb') as fw:
    pickle.dump(df1,fw)
print("Время чтения из pickle файла:")
%time with open(os.path.join(data_folder,"qm9.pkl"), 'rb') as f: df3 = pickle.load(f)

### Для полной достоверности
### Проверим таблицы на идентичность
if df1.equals(df3): print("\nМатрицы из csv и pickle файла идентичны ") 
else: raise Exception("Матрицы не равны")

In [ ]:
### !!! Вредный совет
### Можно сохранять и загружать переменные из кэша Jupyter
### Причем она будет доступна из разных ноутбуков с тем же ядром
### Полезный совет к вредному: выбирайте уникальные имена
### Как в этом примере делит не стоит

### Сохранить в кэш
%store df1
del df1
try: df1
except NameError: print("df1 Deleted successfully") 

### Вытащить их кэша
%store -r df1
try: df1; print("df1 Restored successfully")
except: pass

#### __Выводы по чтению__
Оптимально по соотношению скорость/универсальность - запись и чтение в csv

In [ ]:
### Полезный совет. Не забывайте чистить за собой
### Взглянем сколько весят уже ненужные нам df2 (и столько же df3)
print("Размер df2 равен {:.1f} МБ".format(getsizeof(df1)/1024**2))
### обратите внимаение на функцию format, чем-то похожую printf из Си
### В данном случае это не обязательно, 
### !!! но когда речь про ГБ - УДАЛЯЙТЕ НЕНУЖНОЕ
print("Память текущего процесса ДО чистки    = {:.1f} МБ".format(psutil.Process().memory_info().rss/1024**2))
del df2, df3
print("Память текущего процесса ПОСЛЕ чистки = {:.1f} МБ".format(psutil.Process().memory_info().rss/1024**2))

__ВЫВОД__: Удаляйте большие объекты после использования, особенно на локальных машинах

### 3. Манипуляции с таблицей

In [ ]:
### Взглянем на саму таблицу
### Краткое инфо по колонкам. Сколько не пустых значений и какой у них тип
print(df1.info())
### Простой вызов DataFrame позволяет красиво отрисовывать таблицу с помощью matplotlib
df1

Что же сразу хочется сделать ???  
Правильно: __удалить ненужное__, в нашем случае - это mol_id  
Но с таким количеством данных нужно быть осторожнее и все проверить  
1. Преобразуем mol_id в численный индекс со счетом от нуля.  
Для этого воспользуемся функцией .apply,  которая применят передаваемую ей функцию ко всему набору данных  
Если вы вдруг забыли синтаксис, то всегда можно вызвать помощь с помощью ствроенной функции help
2. Сравним его со стандартной индексацией

In [ ]:
help(df1["mol_id"].apply)

In [ ]:
### Проверка на то, что столбец -- это обычная индексация без проблемных мест
check_val = (df1["mol_id"].apply(lambda x: int(x[4:]) - 1) - np.arange(df1.shape[0])).abs().sum()
print( "mol_id просто индексатор" if check_val==0 else "С mol_id что-то интересное")

In [ ]:
### С чистой совестью можем удалять mol_id
### inplace указывает на то, что мы просим сделать это на месте,
### а не возвращать копию. См. help(df1.drop)
df1.drop(columns="mol_id", inplace=True)
df1

In [ ]:
### Очень важно при отборе фичей для ML избегать сильных корреляций.
### Взглянем на парные корреляции по kendall внутри датасета
corr_map = df1[list(df1)[1:]].corr(method='kendall')
corr_map

In [ ]:
corr_np = np.triu(corr_map.to_numpy(), k=1)
idx = np.unravel_index(corr_np.argmax(), corr_np.shape)
print("Максимальная корреляция между столбцами {} и {} и равна {:.10f}".format( 
            list(corr_map)[idx[0]], list(corr_map)[idx[1]], corr_map.iloc[idx]))

In [ ]:
### Организуем простейший поиск коэффициентов линейного уравнения
from sklearn.linear_model import LinearRegression as LR
### Подгружаем линейный регрессор
lr = LR()
### Оптимизируем линейную модель
### "_ = function_call()" -- удобный способ избавиться от ненужного вывода 
_ = lr.fit(df1["u298"].values.reshape(-1, 1),df1["h298"].values)
print("Итоговое уравнение: H_298 = {:.8E} U_298 + {:.8E}".format(lr.coef_[0], lr.intercept_))
### Глянем на максиммальное и среднее отклонения функции от H_298
print("MAE = {:.3E}, MaxAE = {:.3E}".format((lr.coef_*df1["u298"] - df1["h298"] + lr.intercept_).abs().mean(), 
                                            (lr.coef_*df1["u298"] - df1["h298"] + lr.intercept_).abs().max()) )

Как мы видим столбцы __u298 и h298 скоррелированы с точностью, превыщающей точность первичных данных__.  
Необходимость в столбце h298 отсутсвует, т.к. мы можем его воссоздать с помощью линейного уравнения из u298.  
Мы можем спокойно выкинуть этот столбец.  
Вообще, при построении сложных моделей __обычно отказываются от столбцов при корелляции более 0.98 или 0.95.__  
Так что у нас были бы еще пару кандидатов на вылет.
Отметим, что более многомерные корелляции здесь не проверяются. Не сложно понять, что gap = lumo - homo  
Но такого мы таким простым способом не выявим, к сожалению  
**<u> ДЗ: почистить датасет, чтобы максимальная корреляция по Кендаллу между столбцами была меньше 0.97</u>**

In [ ]:
df1.drop(columns="h298", inplace=True)
df1

#### 3.1 Повыводим всякое разное
1. Найдем 15 строк с минимальным значением дипольного момента, большего 0.15,
   при условии, что r2 строки больше медианого r2 по всему массиву
   и отсортируем их по поляризуемости в убывающем порядке

In [ ]:
### 1 способ. Построчно
### Оставляем толко строки, для которых выполняется условие на r2
df_buff = df1[df1['r2']>df1['r2'].median()]
### Оставляем толко строки, для которых выполняется условие на mu
df_buff = df_buff[df_buff["mu"]>0.15]
### Сортируем таблицу по mu
df_buff = df_buff.sort_values(by='mu')
### Выделяем первые 15 элементов
df_buff = df_buff.head(15)
### Сортируем таблицу по alpha
df_buff = df_buff.sort_values(by="alpha", ascending=False)
df_buff

In [ ]:
### 2 способ. Однострочно
### +: Быстрее писать, код компактнее, в каком-то смысле Python-style coding, кому-то кажется элегантнее
### -: Есть вероятность, что через неделю вы сами потеряется способность разобратсья в своем коде
print("Матрицы, полученные однострочно и многострочно равны" if \
      df1[df1['r2']>df1['r2'].median()][df1["mu"]>0.15].sort_values(by='mu').head(15).sort_values(by="alpha", ascending=False).equals(df_buff) \
      else "Матрицы не равны")
### Даже pandas иногда предупреждает, что вы, возможно, перебарщиваете

2. Выделим только углеводороды по условию,   
   что в smiles не должно быть символов O,N,F в любом регистре
   * С наибольшей теплоемкостью
   * Посмотрим на углеводороды с наибольшей поляризуемостью
   * С наименьшей разницей homo-lumo

In [ ]:
df_hc = df1[df1["smiles"].apply(lambda x: 1 not in [c in x for c in {"o","O","n","N","F"}])]

In [ ]:
arr = df_hc.sort_values(by="cv", ascending=False).head(10)
arr

In [ ]:
### Отрисовываем эти молекул для лучшего восприятия
img=Draw.MolsToGridImage([Chem.MolFromSmiles(x) for x in arr['smiles'].values], molsPerRow=2,subImgSize=(600,450),  
                         legends=["Cv: {:.2f}".format(x) for x in arr['cv'].values])
img

In [ ]:
arr = df_hc.sort_values(by="alpha", ascending=False).head(10)
arr

In [ ]:
img=Draw.MolsToGridImage([Chem.MolFromSmiles(x) for x in arr['smiles'].values], molsPerRow=2,subImgSize=(600,450),  
                         legends=["Alpha: {:.2f}".format(x) for x in arr['alpha'].values])
img

In [ ]:
arr = df_hc.sort_values(by="gap", ascending=True).head(10)
arr

In [ ]:
img=Draw.MolsToGridImage([Chem.MolFromSmiles(x) for x in arr['smiles'].values], molsPerRow=2,subImgSize=(600,450),  
                         legends=["Gap: {:.2E}".format(x) for x in arr['gap'].values])
img

#### 3.2 Цвиттер-ионы
Хоть все молекулы в датасете суммарно незаряжены, может случиться так,  
что стабильная форма -- это цвиттер-ион.  
* Найдем все цвиттер-ионы. Пока без SMARTS.  
  Заряда по модулю больше одного быть не может.   
  "минус" заряд -- это наличие подстроки '-]'  
  "плюс" заряд -- это наличие подстроки '+]'
* Посмотрим максимальное кол-во + и - формальных зарядов
* Для цвиттер-ионов с одним + и - найдем расстояние в связях между зарядами
* Построим зависимость дипольного момента от этого расстояния

In [ ]:
### Считаем количество отр и пол зарядов в каждой молекуле
minus_count = df1['smiles'].apply(lambda x: x.lower().count('-]'))
plus_count  = df1['smiles'].apply(lambda x: x.lower().count('+]'))

### Смотрим на максимальное количество зарядов внутри одной молекулы
print("Максимальное количество '-' зарядов в молекуле = {}".format( minus_count.max()) )
print("Максимальное количество '+' зарядов в молекуле = {}".format( plus_count.max()) )

### Проверяем нейтральность всех молекул
print("Заряженных молекул {}".format((minus_count - plus_count).abs().sum()))

### Выделяем цвиттер-ионы
### Можно выделить подтаблицу по объекту pd.Series с булевыми значениями
### где True означает выбрать этот индекс
df_zi = df1[df1['smiles'].apply(lambda x: True if x.lower().count('-]')==1 else False)].copy()
df_zi

In [ ]:
### Функция на основе rdkit для расчета расстояний между ионами
def GetDistance(smi):
    ### Переводим SMILES в rdkit объект
    m = Chem.MolFromSmiles(smi)
    idx=[]
    ### Проходимся по всем атомам и сохраняем индексы формально заряженных атомов
    for a in m.GetAtoms():
        if a.GetFormalCharge()!=0:
            idx.append(a.GetIdx())
    ### Используя GetDistanceMatrix рассчитваем матрицу расстояний и выделяем нужный элемент
    ### Не самый оптимальный вариант считать всю матрицу для последующего выделения одного элемента
    ### Буду рад, если предложите свое решение
    return Chem.rdmolops.GetDistanceMatrix(m)[idx[0],idx[1]]

**<u> ДЗ: </u>**  
* Выделить датасет только с молекулами, которые содержат более одного фтора   
* Создать столбец со значением максимального расстояния между фторами в молекуле  
* Найти молекулу с самым большим расстоянием между фторами
* Если таких молекул несколько, то найти среди них ту, у которой наибольший дипольный момент


In [ ]:
### Последний столбец )))
df_zi["ion_dist"] = df_zi["smiles"].apply(GetDistance)
df_zi

In [ ]:
### Напишем функцию для отображения 2D гистограмм
### Квантили нужны, чтобы не отображать пустое пространство. Попробуйте quantile=1e-16
def plot2Dhist(df, col1, col2, bins=20, quantile=0.01):
    H, xedges, yedges = np.histogram2d(df[col1].values, df[col2].values, bins=bins, 
                                       range=[df[col1].quantile([quantile, 1-quantile], interpolation='nearest'),
                                              df[col2].quantile([quantile, 1-quantile], interpolation='nearest') ] )
    fig = plt.figure(figsize=(6, 6))
    ax = fig.add_subplot(111, title=col1+"(x) - "+col2+"(y)")
    plt.imshow(H.T, interpolation='nearest', origin='lower', aspect='auto', cmap='plasma',
            extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])

In [ ]:
### Видно небольшой рост наиболее вероятного mu 
###  при увеличении расстояния между ионами
plot2Dhist(df_zi, "ion_dist", "mu", quantile=0.03, bins=[20,50])

In [ ]:
### Сильная корелляция (линейная зависимость)
plot2Dhist(df1, "u298", "g298", quantile=0.01, bins=50)

In [ ]:
### Видна средняя скорелированность
plot2Dhist(df1, "alpha", "cv", quantile=0.005, bins=100)

In [ ]:
### Нет особой корреляции
plot2Dhist(df1, "C", "mu", quantile=0.01, bins=100)

In [ ]:
### Наблюдаем некую квантованность zpve
plot2Dhist(df1, "zpve", "A", quantile=0.03, bins=100)

In [ ]:
### Посмотрим отдельно гистограмму zpve
df1['zpve'].hist(bins=500)

<u> __ДЗ: Попытайтесь объяснить такое поведение zpve__</u>

In [ ]:
### Отрисовываем распределение молекул по числу гетероатомов
fig, ax = plt.subplots(figsize=(12, 6))
a_heights, a_bins = np.histogram(df1['smiles'].apply(lambda x: x.lower().count('n')))
b_heights, b_bins = np.histogram(df1['smiles'].apply(lambda x: x.lower().count('o')), bins=a_bins)
c_heights, c_bins = np.histogram(df1['smiles'].apply(lambda x: x.lower().count('f')), bins=a_bins)
width = (a_bins[1] - a_bins[0])/3
ax.bar(a_bins[:-1], a_heights, width=width, facecolor='cornflowerblue', label="N count")
ax.bar(b_bins[:-1]+width, b_heights, width=width, facecolor='seagreen', label="O count")
ax.bar(c_bins[:-1]-width, c_heights, width=width, facecolor='orange',  label="F count")
_ = ax.legend()

In [ ]:
### Ответ про zpve, не запускать
# df1["atom_count"] = df1['smiles'].apply( lambda x: len(AllChem.AddHs(Chem.MolFromSmiles(x)).GetAtoms()) )
# plot2Dhist(df1, "zpve", "atom_count", quantile=0.01, bins=50)